# Setting Everything Up

In [84]:
import pandas as pd
from datetime import datetime

In [85]:
#import the file from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
from unicodedata import normalize
import re
from datetime import datetime
from pandas import read_csv 
from sklearn.feature_extraction.text import CountVectorizer
import json
from flatten_json import flatten


# I can give a number or use None to remove maximum ceiling & display all columns
pd.options.display.max_columns = None

# # I want to be able to see the entire narrative, so remove the maximum width for each column
# pd.options.display.max_colwidth = None

# pd.options.display.float_format = '{:,.0f}'.format

import string

%matplotlib inline

In [86]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (8, 8)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [88]:
%%R

# My commonly used R imports

require('tidyverse')

In [89]:
#To Download PDFs
from urllib.parse import urljoin

In [90]:
#To import camelot and PDF-related items
import camelot
import ghostscript
import sys

In [91]:
%%R

library(jpeg)
library(wordcloud)
library(RColorBrewer)
library(wordcloud2)
library(tm)


In [92]:
from sklearn.feature_extraction.text import CountVectorizer

In [93]:
%%R
library(vistime)
library(shiny)
library(plotly)
#for timeline plotting

In [94]:
from collections import Counter


# Reading the Data

In [95]:
df_total_tiktoks=pd.read_csv("df_total_tiktoks.csv")
df_total_tiktoks

,number_orig,video_author,video_challenges,video_comments,time_of_creation,video_creator,caption,video_image_post,video_music,video_music_title,num_comments,num_likes,num_views,num_shares,video_tags,trend
0,0,_sheee_gachie,"[LightChallenge(title='_sheee_gachie'), LightC...",[],2023-04-12 12:30:16+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,They called my mum and I had to lie to my mum ...,NaN,id=7212529228053940997 title='original sound' ...,68,320,27600,305700,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221139736071523590 height=1024 width=576 d...,whyyoulying
1,1,francisgaitho,"[LightChallenge(title='whyyoulying'), LightCha...",[],2023-04-09 23:50:36+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Have you seen the #whyyoulying #whyyoulyingcha...,NaN,id=7220201831858703110 title='original sound' ...,277,1311,13300,158200,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7220201803713875205 height=1024 width=576 d...,whyyoulying
2,2,mbiiiish,"[LightChallenge(title='mbiiiish'), LightChalle...",[],2023-04-12 18:16:46+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,@-yiona Tell the people what happened buanaa🤣🤣...,NaN,id=7175709191326190378 title='Were you silent ...,43,143,16200,222900,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221229028756311301 height=1024 width=576 d...,whyyoulying
3,3,lady_marichu,"[LightChallenge(title='whyyoulyingchallenge'),...",[],2023-04-12 13:21:51+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Sometimes I wish I never went there 💀#CapCut #...,NaN,id=7203755811225553670 title='Why you lying by...,267,321,9605,294000,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221153030765382917 height=1024 width=576 d...,whyyoulying
4,4,francisgaitho7,"[LightChallenge(title='844system'), LightChall...",[],2023-04-12 07:57:27+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Do you agree with TikTok’s decision to delete ...,NaN,id=7221069560118774533 title='original sound' ...,221,260,4231,123400,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221069429084343557 height=1024 width=576 d...,whyyoulying
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,66,bydalesmith,"[LightChallenge(title='fyp'), LightChallenge(t...",[],2023-03-25 23:18:44+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut Either way >>>>> 🤤 #whyareyoulyingcha...,NaN,id=7203755811225553670 title='Why you lying by...,0,4,118,1041,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7214627304037780742 height=1024 width=576 d...,whyareyoulying
167,67,chi_herself,"[LightChallenge(title='lazymoneyhacks'), Light...",[],2023-03-20 09:23:57+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Learn how ro make your first 1million naira in...,NaN,id=7203755811225553670 title='Why you lying by...,5,84,203,3082,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7212556758437334277 height=1024 width=576 d...,whyareyoulying
168,68,pappi_doll,[LightChallenge(title='whyareyoulyingchallenge...,[],2023-04-13 18:08:48+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut #fyp #whyareyoulyingchallenge not a h...,NaN,id=7162794517555415854 title='right by my side...,0,7,121,1010,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221598055777258757 height=854 width=480 du...,whyareyoulying
169,69,bubu_x_olly,"[LightChallenge(title='whyareyoulyingoshamo'),...",[],2023-04-18 23:13:23+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,🤭❤️. #oshamoo #whyareyoulyingoshamo #whyareyou...,NaN,id=7203755811225553670 title='Why you lying by...,0,0,130,938,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7223531971970862342 height=1024 width=576 d...,whyareyoulying


# Fetching Link for each Tiktok for analysis

In [100]:
df_total_tiktoks['video_id_for_link']=df_total_tiktoks['video_tags'].str.extract("id=([\d]*) height=")
df_total_tiktoks

,number_orig,video_author,video_challenges,video_comments,time_of_creation,video_creator,caption,video_image_post,video_music,video_music_title,num_comments,num_likes,num_views,num_shares,video_tags,trend,video_id_for_link,link_found
0,0,_sheee_gachie,"[LightChallenge(title='_sheee_gachie'), LightC...",[],2023-04-12 12:30:16+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,They called my mum and I had to lie to my mum ...,NaN,id=7212529228053940997 title='original sound' ...,68,320,27600,305700,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221139736071523590 height=1024 width=576 d...,whyyoulying,7221139736071523590,https://m.tiktok.com/v/7221139736071523590
1,1,francisgaitho,"[LightChallenge(title='whyyoulying'), LightCha...",[],2023-04-09 23:50:36+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Have you seen the #whyyoulying #whyyoulyingcha...,NaN,id=7220201831858703110 title='original sound' ...,277,1311,13300,158200,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7220201803713875205 height=1024 width=576 d...,whyyoulying,7220201803713875205,https://m.tiktok.com/v/7220201803713875205
2,2,mbiiiish,"[LightChallenge(title='mbiiiish'), LightChalle...",[],2023-04-12 18:16:46+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,@-yiona Tell the people what happened buanaa🤣🤣...,NaN,id=7175709191326190378 title='Were you silent ...,43,143,16200,222900,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221229028756311301 height=1024 width=576 d...,whyyoulying,7221229028756311301,https://m.tiktok.com/v/7221229028756311301
3,3,lady_marichu,"[LightChallenge(title='whyyoulyingchallenge'),...",[],2023-04-12 13:21:51+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Sometimes I wish I never went there 💀#CapCut #...,NaN,id=7203755811225553670 title='Why you lying by...,267,321,9605,294000,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221153030765382917 height=1024 width=576 d...,whyyoulying,7221153030765382917,https://m.tiktok.com/v/7221153030765382917
4,4,francisgaitho7,"[LightChallenge(title='844system'), LightChall...",[],2023-04-12 07:57:27+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Do you agree with TikTok’s decision to delete ...,NaN,id=7221069560118774533 title='original sound' ...,221,260,4231,123400,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221069429084343557 height=1024 width=576 d...,whyyoulying,7221069429084343557,https://m.tiktok.com/v/7221069429084343557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,66,bydalesmith,"[LightChallenge(title='fyp'), LightChallenge(t...",[],2023-03-25 23:18:44+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut Either way >>>>> 🤤 #whyareyoulyingcha...,NaN,id=7203755811225553670 title='Why you lying by...,0,4,118,1041,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7214627304037780742 height=1024 width=576 d...,whyareyoulying,7214627304037780742,https://m.tiktok.com/v/7214627304037780742
167,67,chi_herself,"[LightChallenge(title='lazymoneyhacks'), Light...",[],2023-03-20 09:23:57+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Learn how ro make your first 1million naira in...,NaN,id=7203755811225553670 title='Why you lying by...,5,84,203,3082,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7212556758437334277 height=1024 width=576 d...,whyareyoulying,7212556758437334277,https://m.tiktok.com/v/7212556758437334277
168,68,pappi_doll,[LightChallenge(title='whyareyoulyingchallenge...,[],2023-04-13 18:08:48+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut #fyp #whyareyoulyingchallenge not a h...,NaN,id=7162794517555415854 title='right by my side...,0,7,121,1010,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221598055777258757 height=854 width=480 du...,whyareyoulying,7221598055777258757,https://m.tiktok.com/v/7221598055777258757
169,69,bubu_x_olly,"[LightChallenge(title='whyareyoulyingoshamo'),...",[],2023-04-18 23:13:23+00:00,<tiktokapipy.async_api

In [101]:
import asyncio

import numpy as np
import pandas as pd
from tiktokapipy.async_api import AsyncTikTokAPI

def video_link(video_id: int) -> str:
    """Get a working link to a TikTok video from the video's unique id."""
    return f"https://m.tiktok.com/v/{video_id}"
            
    
df_total_tiktoks ['link_found'] =  df_total_tiktoks.video_id_for_link.apply(video_link)
df_total_tiktoks ['link_found']


0      https://m.tiktok.com/v/7221139736071523590
1      https://m.tiktok.com/v/7220201803713875205
2      https://m.tiktok.com/v/7221229028756311301
3      https://m.tiktok.com/v/7221153030765382917
4      https://m.tiktok.com/v/7221069429084343557
                          ...                    
166    https://m.tiktok.com/v/7214627304037780742
167    https://m.tiktok.com/v/7212556758437334277
168    https://m.tiktok.com/v/7221598055777258757
169    https://m.tiktok.com/v/7223531971970862342
170    https://m.tiktok.com/v/7215271877067984133
Name: link_found, Length: 171, dtype: object

In [102]:
df_total_tiktoks.to_csv("df_tiktok_with_links.csv")

In [103]:
df_total_tiktoks

,number_orig,video_author,video_challenges,video_comments,time_of_creation,video_creator,caption,video_image_post,video_music,video_music_title,num_comments,num_likes,num_views,num_shares,video_tags,trend,video_id_for_link,link_found
0,0,_sheee_gachie,"[LightChallenge(title='_sheee_gachie'), LightC...",[],2023-04-12 12:30:16+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,They called my mum and I had to lie to my mum ...,NaN,id=7212529228053940997 title='original sound' ...,68,320,27600,305700,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221139736071523590 height=1024 width=576 d...,whyyoulying,7221139736071523590,https://m.tiktok.com/v/7221139736071523590
1,1,francisgaitho,"[LightChallenge(title='whyyoulying'), LightCha...",[],2023-04-09 23:50:36+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Have you seen the #whyyoulying #whyyoulyingcha...,NaN,id=7220201831858703110 title='original sound' ...,277,1311,13300,158200,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7220201803713875205 height=1024 width=576 d...,whyyoulying,7220201803713875205,https://m.tiktok.com/v/7220201803713875205
2,2,mbiiiish,"[LightChallenge(title='mbiiiish'), LightChalle...",[],2023-04-12 18:16:46+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,@-yiona Tell the people what happened buanaa🤣🤣...,NaN,id=7175709191326190378 title='Were you silent ...,43,143,16200,222900,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221229028756311301 height=1024 width=576 d...,whyyoulying,7221229028756311301,https://m.tiktok.com/v/7221229028756311301
3,3,lady_marichu,"[LightChallenge(title='whyyoulyingchallenge'),...",[],2023-04-12 13:21:51+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Sometimes I wish I never went there 💀#CapCut #...,NaN,id=7203755811225553670 title='Why you lying by...,267,321,9605,294000,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221153030765382917 height=1024 width=576 d...,whyyoulying,7221153030765382917,https://m.tiktok.com/v/7221153030765382917
4,4,francisgaitho7,"[LightChallenge(title='844system'), LightChall...",[],2023-04-12 07:57:27+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Do you agree with TikTok’s decision to delete ...,NaN,id=7221069560118774533 title='original sound' ...,221,260,4231,123400,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221069429084343557 height=1024 width=576 d...,whyyoulying,7221069429084343557,https://m.tiktok.com/v/7221069429084343557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,66,bydalesmith,"[LightChallenge(title='fyp'), LightChallenge(t...",[],2023-03-25 23:18:44+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut Either way >>>>> 🤤 #whyareyoulyingcha...,NaN,id=7203755811225553670 title='Why you lying by...,0,4,118,1041,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7214627304037780742 height=1024 width=576 d...,whyareyoulying,7214627304037780742,https://m.tiktok.com/v/7214627304037780742
167,67,chi_herself,"[LightChallenge(title='lazymoneyhacks'), Light...",[],2023-03-20 09:23:57+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Learn how ro make your first 1million naira in...,NaN,id=7203755811225553670 title='Why you lying by...,5,84,203,3082,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7212556758437334277 height=1024 width=576 d...,whyareyoulying,7212556758437334277,https://m.tiktok.com/v/7212556758437334277
168,68,pappi_doll,[LightChallenge(title='whyareyoulyingchallenge...,[],2023-04-13 18:08:48+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut #fyp #whyareyoulyingchallenge not a h...,NaN,id=7162794517555415854 title='right by my side...,0,7,121,1010,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221598055777258757 height=854 width=480 du...,whyareyoulying,7221598055777258757,https://m.tiktok.com/v/7221598055777258757
169,69,bubu_x_olly,"[LightChallenge(title='whyareyoulyingoshamo'),...",[],2023-04-18 23:13:23+00:00,<tiktokapipy.async_api

In [104]:
df_tiktok_no_duplicates=pd.read_csv("df_tiktok_no_duplicates.csv")

In [105]:
df_tiktok_no_duplicates

,Unnamed: 0,number_orig,video_author,video_challenges,video_comments,time_of_creation,video_creator,caption,video_image_post,video_music,video_music_title,num_comments,num_likes,num_views,num_shares,video_tags,trend,video_id_for_link,link_found,bullied_by,bullied_about_1,bullied_about_2,bullied_about_3
0,0,0,_sheee_gachie,"[LightChallenge(title='_sheee_gachie'), LightC...",[],2023-04-12 12:30:16+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,They called my mum and I had to lie to my mum ...,NaN,id=7212529228053940997 title='original sound' ...,68,320,27600,305700,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221139736071523590 height=1024 width=576 d...,whyyoulying,7221139736071523590,https://m.tiktok.com/v/7221139736071523590,NaN,NaN,NaN,NaN
1,3,3,lady_marichu,"[LightChallenge(title='whyyoulyingchallenge'),...",[],2023-04-12 13:21:51+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Sometimes I wish I never went there 💀#CapCut #...,NaN,id=7203755811225553670 title='Why you lying by...,267,321,9605,294000,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221153030765382917 height=1024 width=576 d...,whyyoulying,7221153030765382917,https://m.tiktok.com/v/7221153030765382917,NaN,NaN,NaN,NaN
2,2,2,mbiiiish,"[LightChallenge(title='mbiiiish'), LightChalle...",[],2023-04-12 18:16:46+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,@-yiona Tell the people what happened buanaa🤣🤣...,NaN,id=7175709191326190378 title='Were you silent ...,43,143,16200,222900,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221229028756311301 height=1024 width=576 d...,whyyoulying,7221229028756311301,https://m.tiktok.com/v/7221229028756311301,NaN,NaN,NaN,NaN
3,1,1,francisgaitho,"[LightChallenge(title='whyyoulying'), LightCha...",[],2023-04-09 23:50:36+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Have you seen the #whyyoulying #whyyoulyingcha...,NaN,id=7220201831858703110 title='original sound' ...,277,1311,13300,158200,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7220201803713875205 height=1024 width=576 d...,whyyoulying,7220201803713875205,https://m.tiktok.com/v/7220201803713875205,NaN,NaN,NaN,NaN
4,4,4,francisgaitho7,"[LightChallenge(title='844system'), LightChall...",[],2023-04-12 07:57:27+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,Do you agree with TikTok’s decision to delete ...,NaN,id=7221069560118774533 title='original sound' ...,221,260,4231,123400,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221069429084343557 height=1024 width=576 d...,whyyoulying,7221069429084343557,https://m.tiktok.com/v/7221069429084343557,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,71,71,missjoyous1,"[LightChallenge(title='kenyahighschools'), Lig...",[],2023-04-13 10:23:41+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,"my school mates mnaniangusha,nataka nichangamk...",NaN,id=7221478196867959557 title='original sound' ...,0,6,33,217,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7221478200168221957 height=1024 width=576 d...,whyyoulying,7221478200168221957,https://m.tiktok.com/v/7221478200168221957,NaN,NaN,NaN,NaN
136,85,85,_chrisben_official,"[LightChallenge(title='goodvibes'), LightChall...",[],2023-04-17 10:52:39+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#chrisben #storytime #viral #kenyantiktok #tre...,NaN,id=7166039999551375362 title='Badda Dan' play_...,0,2,25,206,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7222970001450388741 height=1024 width=576 d...,whyyoulying,7222970001450388741,https://m.tiktok.com/v/7222970001450388741,NaN,NaN,NaN,NaN
137,98,98,modunni2,"[LightChallenge(title='capcut'), LightChalleng...",[],2023-04-11 17:31:39+00:00,<tiktokapipy.async_api.AsyncLightUserGetter ob...,#CapCut #whyyoulyingchallenge #viral 😂😂❤️,NaN,id=7203755811225553670 title='Why you lying by...,0,6,19,205,<tiktokapipy.async_api.AsyncLightChallengeIter...,id=7220846320318205189 height=1022 widt